# Script to clean the data extracted from MIMIC, and save clean version for future use.

- Remove clearly anomalous values.
- Sum GCS components, remove individual components, and introduce GCS_total.
- Convert units for consisitency with GICU (UK data).
- Introduce 'airway' variable (remove PEEP and AIRWAY)...this is a reduced version of current AIRWAY

*Note: in the following mapping we exclude venous po2 and pco2, and diastolic bp.

In [12]:
variable_mapping = dict()
variable_mapping['fio2'] = [226754, 227009, 227010,223835]
variable_mapping['resp'] = [220210, 224688, 224689, 224690]
variable_mapping['po2'] = [226770,227039,220224] 
variable_mapping['pco2'] = [220235,227036]  
variable_mapping['temp'] = [223761, 223762, 224027] 
variable_mapping['hr'] = [220045]
variable_mapping['bp'] = [220050, 220059, 220179, 224167, 225309, 227243, 226850, 226852]
variable_mapping['k'] = [220640, 227464, 227442, 226772, 226535]
variable_mapping['na'] = [220645, 226534, 226776]
variable_mapping['hco3'] = [224826, 226759, 227443]
variable_mapping['spo2'] = [220227, 220277, 226860,226861,226862,226863,226865,228232]
variable_mapping['bun'] = [225624, 227000, 227001]
variable_mapping['airway'] = [223838, 224832, 224391, 227810,223837, 224829]
variable_mapping['gcs'] = [220739, 223900, 223901, 226755, 226756, 226757, 226758, 227011, 227012, 227013, 227014,228112]
variable_mapping['creatinine'] = [220615, 226752, 227005]
variable_mapping['pain'] = [223791, 227881]
variable_mapping['urine'] = [227519, 227059]
variable_mapping['haemoglobin'] = [220228]
variable_mapping['peep'] = [220339, 224699, 224700]

In [13]:
inv_var_mapping = {itemid: var for var, itemid_list in variable_mapping.items()
                               for itemid in itemid_list}

We will use the following dictionary of tuples to check for non-physical variable values:

In [10]:
physical_limit = dict()
physical_limit['creatinine'] = (0.0, 50.0)
physical_limit['fio2'] = (0.0, 100.0)     
physical_limit['resp'] = (0.0,100.0)      
physical_limit['po2'] = (0.0,500.0)
physical_limit['pco2'] = (0.0,500.0) 
physical_limit['temp'] = (80.0, 120.0)    
physical_limit['hr'] = (0.0, 250.0)  
physical_limit['bp'] = (0.0, 500.0) 
physical_limit['k'] = (0.0, 50.0)   
physical_limit['na'] = (0.0, 500.0)  
physical_limit['hco3'] = (0,100.0)    
physical_limit['spo2'] = (10.0, 100.0) 
physical_limit['bun'] = (0.0,300.0)  
physical_limit['gcs'] = (0.0, 16.0)  
physical_limit['pain'] = (0.0, 10.0) 
physical_limit['peep'] = (0.0, 50.0)          
physical_limit['haemoglobin'] = (0.0, 100.0)  
physical_limit['airway'] = (None,None)

In [3]:
def _remove_anomalies(data, physical_limit, variable_dict, verbose=False):
    
    ## define which ITEMIDs correspond to which variables.  
    get_var = lambda it: [variable for variable,items in variable_dict.items() if it in items][0]
    all_itemids = [item for sublist in variable_dict.values() for item in sublist]
    var_id = {item: get_var(item) for item in all_itemids}
    
    data['remove_col'] = data.apply(lambda row: 1 if row['C.VALUENUM']>physical_limit[var_id[row['D.ITEMID']]][1] or row['C.VALUENUM']<physical_limit[var_id[row['D.ITEMID']]][0] else 0, axis=1)
    
    ## now filter the data frame:
    print( "%d rows to remove:" %sum(data['remove_col']==1))
    
    if verbose:
        print( "For example:")
        removed = data[data['remove_col']==1].sample(0.001)
        removed.print_rows(num_rows=20)
    
    data = data[data['remove_col']==0]
    data = data.drop('remove_col', axis=1)
    print("Anomalies removed.")
    return data

#### Load data and remove anomalies: 

In [6]:
from google.cloud import bigquery
# Construct a BigQuery client object.
client = bigquery.Client()

In [ ]:
from google.cloud import bigquery_storage_v1beta1

In [4]:
import pandas as pd
data2 = pd.read_pickle('final_interesting_chartevents.pickle')

In [7]:
print(len(data2))
data3 = data2[~data2['D.ITEMID'].isin([226062,226063,227516,228151])]
print( len(data3))
del data3

10814977
10807178


In [15]:
data2 = data2[~data2['D.ITEMID'].isin([226062,226063,227516,228151])]


In [16]:
data2['var'] = data2['D.ITEMID'].apply(lambda row: inv_var_mapping[row]) 

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
len(data2[data2['var']=='airway'])

364402

In [19]:
data2['lower_limit'] = data2['var'].apply(lambda row: physical_limit[row][0])
data2['upper_limit'] = data2['var'].apply(lambda row: physical_limit[row][1])

/usr/lib/python3/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [9]:
data2[data2['C.VALUENUM'].isnull()]['D.ITEMID'].unique(), len(data2[data2['C.VALUENUM'].isnull()])

(array([224829, 224391, 224027, 224832, 223838, 223837]), 589151)

In [22]:
data2 = data2[(data2['C.VALUENUM'].isnull() & (data2['var']!='airway')) 
            | (data2['C.VALUENUM'] < data2['lower_limit'])
            | (data2['C.VALUENUM'] > data2['upper_limit']) ]

247235

We add column to data that says what type of variable it is (i.e. the variable name):

In [23]:
data2 = data2.rename(columns={'var': 'VARIABLE'})

In [25]:
data2.to_pickle('cleaning_anamoly.pickle')

#### We now sum GCS components:  
(And remove those that do not have all 3 cpts measured)

In [1]:
import pandas as pd
all_data = pd.read_pickle('cleaning_anamoly.pickle')

In [32]:
all_data.keys()

Index(['C.SUBJECT_ID', 'C.HADM_ID', 'C.ICUSTAY_ID', 'D.ITEMID', 'C.CHARTTIME',
       'C.VALUE', 'C.VALUENUM', 'C.VALUEUOM', 'II.INTIME', 'II.OUTTIME',
       'II.LOS', 'D.LABEL', 'D.UNITNAME', 'hrs_bd', 'final_4hr', 'final_24hr',
       'cohort', 'in_h_death', 'in_icu_death', 'readmit', 'outcome',
       'VARIABLE', 'lower_limit', 'upper_limit'],
      dtype='object')

In [2]:
gcs_summed = all_data[all_data['D.ITEMID'].isin(variable_mapping['gcs'])]
gcs_summed['ncpts'] = gcs_summed['C.VALUENUM']
gcs_summed = gcs_summed.groupby(['C.ICUSTAY_ID', 'C.CHARTTIME'])

NameError: name 'variable_mapping' is not defined

In [ ]:
gcs_summed = gcs_summed.aggregate({'C.VALUENUM': 'sum', 
                     'ncpts': 'count',
                     'hrs_bd': 'min',
                     'C.HADM_ID':'min',
                     'C.SUBJECT_ID':'min',
                     'D.ITEMID':'min',
                     'C.VALUE':'min',
                     'C.VALUEUOM':'min',
                     'D.LABEL':'min',
                     'D.UNITNAME':'min',
                     'II.INTIME':'min',
                     'II.LOS':'min',
                     'II.OUTTIME':'min',
                     'final_4hr':'min',
                     'final_24hr':'min',
                     'cohort':'min',
                     'in_h_death':'min',
                     'in_icu_death':'min',
                     'readmit':'min',
                     'outcome':'min',
                     'VARIABLE':'min',
                     'lower_limit':'min',
                     'upper_limit':'min'
                    })

About 1% of GCS measures are not 'full' (i.e. one or more cpt missing)
We - remove these because it would be unfair to test on two components without altering the test.

In [ ]:
_frac_incomplete = sum(gcs_summed['ncpts']<3)/float(len(gcs_summed))
print("%.4f of gcs measures not complete." %_frac_incomplete)
gcs_summed = gcs_summed[gcs_summed['ncpts']==3]

#### Remove all single GCS components and append the complete GCS values to the data:

In [ ]:
all_data = all_data[~all_data['D.ITEMID'].isin(variable_mapping['gcs'])]

In [ ]:
gcs_summed = gcs_summed.reset_index()

In [ ]:
gcs_summed = gcs_summed.drop(columns='ncpts')
all_data = all_data.append(gcs_summed, ignore_index=True)
## VARIABLE='GCS' is now GCS total

In [ ]:
all_data.tail(10)

#### And convert all required units to be consistent with our UK data:

In [15]:
units_to_convert = ['creatinine', 'temp', 'po2', 'pco2', 'bun']

conv_crea = lambda crea: 88.42 * crea  ## convert from mg/dL (MIMIC) to umol/L (GICU)
conv_temp = lambda temp: (temp-32)/1.8 ## convert from Farenhiet (MIMIC) to Celcius (GICU)
conv_bg = lambda gas: 0.1333 * gas   ## convert blod gas from mmHg (MIMIC) to kPa (GICU)
conv_bun = lambda bun: 0.3571 * bun  ## convert mg/dL (MIMC) to mmol/L (GICU)

def _convert(var, val):
    
    new_val = None
    if var=='creatinine':
        new_val = conv_crea(val)
    elif var=='temp':
        if val==None:
            new_val=None
        else:
            new_val = conv_temp(val)
    elif var=='po2' or var=='pco2':
        new_val = conv_bg(val)
    elif var=='bun':
        new_val = conv_bun(val)
        
    return new_val

In [9]:
import numpy as np
all_data['C.VALUENUM'] = np.where(all_data['VARIABLE']=='creatinine',all_data['C.VALUENUM']*88.42, all_data['C.VALUENUM'])
all_data['C.VALUENUM'] = np.where(all_data['VARIABLE']=='temp', (all_data['C.VALUENUM']-32)/1.8, all_data['C.VALUENUM'])
all_data['C.VALUENUM'] = np.where(all_data['VARIABLE'].isin(['po2','pco2']), all_data['C.VALUENUM']*0.1333, all_data['C.VALUENUM'])
all_data['C.VALUENUM'] = np.where(all_data['VARIABLE']=='bun',all_data['C.VALUENUM']*0.3571, all_data['C.VALUENUM'])

In [8]:
all_data = all_data.drop(columns='C.VALUENUM2')

In [10]:
all_data.to_pickle('cleaning_gcs.pickle')

#### We now work out if airway is patent...

- We simply use presence of ETT as proxy for non-patent airway. 
- remove PEEP (it would be possible to stipulate PEEP + ETT -> non-patent, but simultaneity calculation is an unecessary complication).

In [14]:
airway_reduced = all_data[all_data['D.ITEMID'].isin(variable_mapping['airway'])].groupby(['C.ICUSTAY_ID', 'C.CHARTTIME'])


In [15]:
airway_reduced = airway_reduced.aggregate({
                     'hrs_bd': 'min',
                     'C.HADM_ID':'min',
                     'C.SUBJECT_ID':'min',
                     'D.ITEMID':'min',
                     'C.VALUE':'min',
                     'C.VALUEUOM':'min',
                     'D.LABEL':'min',
                     'D.UNITNAME':'min',
                     'II.INTIME':'min',
                     'II.LOS':'min',
                     'II.OUTTIME':'min',
                     'final_4hr':'min',
                     'final_24hr':'min',
                     'cohort':'min',
                     'in_h_death':'min',
                     'in_icu_death':'min',
                     'readmit':'min',
                     'outcome':'min',
                     'VARIABLE':'min',
                     'lower_limit':'min',
                     'upper_limit':'min'
                    })

In [23]:
airway_reduced = airway_reduced.reset_index()

In [26]:
airway_reduced['C.VALUENUM'] = 1.0

#### Remove all PEEP and AIRWAY:

In [27]:
all_data = all_data[~all_data['D.ITEMID'].isin(variable_mapping['airway']+variable_mapping['peep'])]

#### Add reduced airway variable back in and save this 'clean' version of the data:

In [29]:
all_data = all_data.append(airway_reduced, ignore_index=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [32]:
all_data.to_pickle('all_cleaned.pickle')

#### We now extract the CALLOUT times (RFD flags) for each icustay and add these to the data: 

For each ICUSTAY want to get the corresponding successful CALLOUTs (only those with outcomes marked as 'Discharged'). There are a small number of stays with mutliple succesful discharges, we remove these instances from the data to avoid confusion (they correspond the patients being transfered between different ICUs).

In [22]:
all_data = pd.read_pickle('all_cleaned.pickle')

In [33]:
from google.cloud import bigquery
# Construct a BigQuery client object.
client = bigquery.Client()

In [34]:
query_job = 'SELECT * from test.callout'
callouts = client.query(query_job).to_dataframe()

In [39]:
_stays = all_data.groupby(by='C.ICUSTAY_ID').aggregate({'C.HADM_ID':'min', 'II.INTIME':'min', 'II.OUTTIME':'min'}).reset_index()

In [45]:
_stays = _stays.rename(columns={'C.HADM_ID': 'HADM_ID', 'II.INTIME':'IN', 'II.OUTTIME':'OUT'})

In [49]:
_stays.head(10)

,C.ICUSTAY_ID,HADM_ID,IN,OUT
0,200001,152234,2181-11-25 19:06:12,2181-11-28 20:59:25
1,200010,192256,2132-08-04 23:03:19,2132-08-05 22:14:11
2,200011,121562,2188-08-06 01:39:24,2188-08-07 16:50:53
3,200016,117458,2150-12-02 15:59:20,2150-12-03 14:54:29
4,200021,109307,2114-12-26 19:45:12,2114-12-27 22:46:28
5,200024,179633,2127-03-03 16:09:07,2127-03-04 01:18:06
6,200033,198650,2198-08-07 17:56:17,2198-08-21 14:59:18
7,200034,145866,2186-01-20 10:26:04,2186-01-23 15:38:36
8,200038,175436,2143-10-24 20:35:24,2143-10-25 22:00:02
9,200049,149216,2118-08-28 08:56:44,2118-08-29 19:02:09


In [50]:
_stays_join = _stays.merge(callouts, on='HADM_ID',how='inner')

In [57]:
_stays_join['RFD'] = _stays_join['CREATETIME'].apply(lambda ti: ti.replace(tzinfo=None))

In [58]:
_stays_join.head(10)

,C.ICUSTAY_ID,HADM_ID,IN,OUT,ROW_ID,SUBJECT_ID,SUBMIT_WARDID,SUBMIT_CAREUNIT,CURR_WARDID,CURR_CAREUNIT,...,CALLOUT_OUTCOME,DISCHARGE_WARDID,ACKNOWLEDGE_STATUS,CREATETIME,UPDATETIME,ACKNOWLEDGETIME,OUTCOMETIME,FIRSTRESERVATIONTIME,CURRENTRESERVATIONTIME,RFD
0,200001,152234,2181-11-25 19:06:12,2181-11-28 20:59:25,20939,55973,23.0,None,54.0,MICU,...,Discharged,54.0,Acknowledged,2181-11-28 11:16:36,2181-11-28 11:16:36,2181-11-28 11:34:05,2181-11-28 21:10:25,2181-11-28 18:10:26,NaT,2181-11-28 11:16:36
1,200010,192256,2132-08-04 23:03:19,2132-08-05 22:14:11,4647,11861,50.0,None,4.0,MICU,...,Discharged,4.0,Acknowledged,2132-08-05 10:47:14,2132-08-05 14:40:19,2132-08-05 15:20:39,2132-08-05 22:29:30,2132-08-05 18:44:21,NaT,2132-08-05 10:47:14
2,200021,109307,2114-12-26 19:45:12,2114-12-27 22:46:28,22732,61691,33.0,None,31.0,SICU,...,Discharged,31.0,Acknowledged,2114-12-27 14:37:41,2114-12-27 14:37:41,2114-12-27 15:22:35,2114-12-27 23:10:24,2114-12-27 21:55:24,NaT,2114-12-27 14:37:41
3,200024,179633,2127-03-03 16:09:07,2127-03-04 01:18:06,27327,76603,52.0,None,45.0,MICU,...,Discharged,45.0,Acknowledged,2127-02-27 12:48:34,2127-02-27 12:48:34,2127-02-27 13:11:32,2127-02-28 14:10:30,2127-02-27 16:10:26,NaT,2127-02-27 12:48:34
4,276949,179633,2127-02-25 03:37:49,2127-02-28 14:00:21,27327,76603,52.0,None,45.0,MICU,...,Discharged,45.0,Acknowledged,2127-02-27 12:48:34,2127-02-27 12:48:34,2127-02-27 13:11:32,2127-02-28 14:10:30,2127-02-27 16:10:26,NaT,2127-02-27 12:48:34
5,200033,198650,2198-08-07 17:56:17,2198-08-21 14:59:18,21079,56369,33.0,SICU,33.0,SICU,...,Cancelled,NaN,Revised,2198-08-10 10:17:16,2198-08-10 18:10:58,NaT,2198-08-10 18:10:58,NaT,NaT,2198-08-10 10:17:16
6,200034,145866,2186-01-20 10:26:04,2186-01-23 15:38:36,33991,98276,57.0,None,31.0,SICU,...,Discharged,31.0,Acknowledged,2186-01-23 10:09:21,2186-01-23 10:09:21,2186-01-23 10:21:22,2186-01-23 15:55:20,2186-01-23 11:25:23,NaT,2186-01-23 10:09:21
7,200038,175436,2143-10-24 20:35:24,2143-10-25 22:00:02,13210,29708,50.0,None,31.0,MICU,...,Discharged,31.0,Acknowledged,2143-10-25 17:57:14,2143-10-25 17:57:14,2143-10-25 18:41:18,2143-10-25 22:10:04,NaT,NaT,2143-10-25 17:57:14
8,200049,149216,2118-08-28 08:56:44,2118-08-29 19:02:09,26268,73241,23.0,None,54.0,MICU,...,Discharged,54.0,Acknowledged,2118-08-29 15:49:16,2118-08-29 16:56:15,2118-08-29 16:56:17,2118-08-29 19:26:12,2118-08-29 17:10:24,NaT,2118-08-29 15:49:16
9,200049,149216,2118-08-28 08:56:44,2118-08-29 19:02:09,26267,73241,50.0,None,54.0,MICU,...,Discharged,54.0,Acknowledged,2118-08-17 09:34:04,2118-08-17 11:17:46,2118-08-17 11:37:50,2118-08-17 18:11:08,2118-08-17 14:40:28,NaT,2118-08-17 09:34:04


In [59]:
_stays_join['callout_match'] = _stays_join.apply(lambda row: 1 if (row['RFD']>=row['IN'] and row['RFD']<=row['OUT']) else 0, axis=1)

In [63]:
_stays_join = _stays_join[(_stays_join['callout_match']==1) & (_stays_join['CALLOUT_OUTCOME']=='Discharged')]

In [70]:
counts = _stays_join.groupby(by='C.ICUSTAY_ID').aggregate({'RFD':'count'})
counts = counts.rename(columns={'RFD': 'count'}).reset_index()

In [67]:
print( "There are %d stays with more than one successful CALLOUT (i.e. transfers)." %sum(counts['count']>1))

There are 208 stays with more than one successful CALLOUT (i.e. transfers).


#### Remove these stays: 

In [72]:
remove_stays = counts[counts['count']>1]['C.ICUSTAY_ID']

In [77]:
_stays_join = _stays_join[~_stays_join['C.ICUSTAY_ID'].isin(remove_stays)]

In [78]:
print( "Of the original %d icu stays," %len(all_data['C.ICUSTAY_ID'].unique()))
print( "%d remain (have successful single callouts)" %len(_stays_join['C.ICUSTAY_ID'].unique()))

Of the original 14432 icu stays,
11288 remain (have successful single callouts)


Note: successful discharge here is different to "positive outcome" follwoing discharge. Successful simply means the patient did actually leave the icu (i.e. the callout was acted upon) 

#### We now join the CALLOUTS (RFD flags) to the original data:

In [80]:
_stays_join2 = _stays_join[['C.ICUSTAY_ID', 'RFD']]
all_data = all_data.merge(_stays_join2, how='inner', on='C.ICUSTAY_ID')

#### We now add a column 'hrs_bRFD':

This is similar to the column 'hrs_bd' which we added previoulsy. We will filter on this column later to construct the feature matrix.

In [81]:
all_data['hrs_bRFD'] = all_data['RFD'] - all_data['C.CHARTTIME']


In [82]:
all_data['hrs_bRFD'] = all_data['hrs_bRFD'].apply(lambda row: row.total_seconds() / float(60**2))

In [83]:
all_data.to_pickle('all_cleaned_with_callouts.pickle')

In [84]:
print( "There are a total of %d rows in the cleaned data." %len(all_data))

There are a total of 6133161 rows in the cleaned data.


In [29]:
print( "There are a total of %d rows in the cleaned data." %len(all_data))

There are a total of 5495513 rows in the cleaned data.
